# Типы данных: Словари и Множества

На этой лекции мы закончим разбирать встроенные типы данных в Python и поговорим о словарях (dict) и множествах (set, frozenset).

## Словари

### Знакомство со словарями

Словари являются неупорядоченными изменяемыми коллекциями пар элементов вида ключ-значение. О словарях можно думать, как об ассоциативных массивах, т.е. о списках, чьи индексы - ключи, на первый взгляд, являются произвольными объектами. Так ключом словаря может быть объект типа int, float, str, frozenset, функция или даже объект пользовательского типа данных. Однако не все так просто, ибо на объекты, которым разрешено выступать в роли ключей, накладывается одно важное ограничение - объекты должны быть хешируемыми. Что это означает? А означает это следующее:

- Значение [хэш-функции](https://ru.wikipedia.org/wiki/%D0%A5%D0%B5%D1%88-%D1%84%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D1%8F) объекта должно быть определено, и не изменяться на протяжении всего времени выполнения программы. Подробнее о хэш функциях вам расскажут в курсе Алгоритмы и Структуры Данных, сейчас хэш функцию можно воспринимать, как некоторый черный ящик, которыей принимает на вход некоторый объект и ставит в соответствие ему последовательность бит фиксированной длины - т.е. число из фиксированного диапазона. Для того, чтобы объект Python был хэшируемым необходимо, чтобы в нем была реализована специальная функция `__hash__()`, т.е. чтобы интерпретатор мог выполнять вызов встроенной функции `hash()` с вашим объектом;  
- Объект должен поддерживать использование оператора ==;
- Если два объекта равны: obj1 == obj2, то должно выполняться равенство: hash(obj1) == hash(obj2); 

Если все три условия выполнены, то объект считается хэшируемым и может быть использован в качестве ключа. Числовые типы данных, строки, замороженные множества всегда хэшируемы. Если все элементы кортежа хэшируемы, то и кортеж является хэшируемым:

In [6]:
objects = [1, 3.14, 'string', frozenset((1, 2, 3))]

for obj in objects:
    print(
        f'object value: {obj}; object type: {type(obj).__name__}; '
        f'hash function value: {hash(obj)};'
    )

tuple_hashable = tuple(objects)
print(f'hashable tuple hash: {hash(tuple_hashable)}')

# TypeError
tuple_unhashable = (0, [1])
print(hash(tuple_unhashable))

object value: 1; object type: int; hash function value: 1;
object value: 3.14; object type: float; hash function value: 322818021289917443;
object value: string; object type: str; hash function value: -7057234837228677891;
object value: frozenset({1, 2, 3}); object type: frozenset; hash function value: -272375401224217160;
hashable tuple hash: -6984807964728689118


TypeError: unhashable type: 'list'

Ключи словря являются уникальными. Попытка добавления пары с существующим ключом приведет к изменению значения по этому ключу. На объекты, которые хранятся в словаре - значения, - никаких ограничений не накладывается.

Давайте рассмотрим создание словарей в коде:

In [19]:
dicts = [
    {'x':42, 'y':3.14, 'z':7},
    {1:2, 3:4},
    {1:'za', 'br':23},
    {},
    dict(x=42, y=3.14, z=7),
    dict([(1, 2), (3, 4)]),
    dict([(1,'za'), ('br',23)]),
    dict(),
    dict([(letter, ord(letter)) for letter in 'abcde']),
    {letter: ord(letter) for letter in 'abcde'},
    {
        letter: ord(letter) for letter in 'abcde'
        if ord(letter) % 2
    },
    dict([('x', 1), ('y', 2), ('x', 126), ('y', 365)]),
    dict.fromkeys('abc', 2),
    dict.fromkeys('abc')
]

In [20]:
print('CREATED DICTS'.center(80, '='))
for dict_example in dicts:
    print(f'{type(dict_example).__name__}: {dict_example};')

print(''.center(80, '='))

=================================CREATED DICTS==================================
dict: {'x': 42, 'y': 3.14, 'z': 7};
dict: {1: 2, 3: 4};
dict: {1: 'za', 'br': 23};
dict: {};
dict: {'x': 42, 'y': 3.14, 'z': 7};
dict: {1: 2, 3: 4};
dict: {1: 'za', 'br': 23};
dict: {};
dict: {'a': 97, 'b': 98, 'c': 99, 'd': 100, 'e': 101};
dict: {'a': 97, 'b': 98, 'c': 99, 'd': 100, 'e': 101};
dict: {'a': 97, 'c': 99, 'e': 101};
dict: {'x': 126, 'y': 365};
dict: {'a': 2, 'b': 2, 'c': 2};
dict: {'a': None, 'b': None, 'c': None};


### Словари и методы последовательностей или "Почему словари не последовательности"

Почти все операции над последовательностями, кроме конкатенации и повторения (подумайте, почему), применима и к словарям. Но тут есть свои нюансы.

**len()**:

Функция `len()`, работает вполне понятным образом - определяет количество пар ключ-значение, которое хранится в словаре:

In [21]:
my_dict = {'a': 1, 'b': 2}

print(f'dict len: {len(my_dict)}')

dict len: 2


**Оператор in**:

Словари поддерживают оператор `in`, однако в отличие от последовательностей, метод in проверяет вхождение объекта в словарь не в качестве элемента, а в качестве ключа. Т.е. in будет возвращать True только в том случае, если в словаре сожержиться ключ, с тем же значением, что и аргумент оператора:

In [22]:
my_dict = {'a': 1, 'b': 2}

print('a' in my_dict)
print('c' in my_dict)

True
False


**__getitem__()**

Также словари поддерживают доступ к элементам через квадратные скобки. Аргументом здесь является объект-ключ. Если словарь содержит элемент с данным ключом, то результатом выполнения данного обращения будет являться этот хранящийся ссылка на хранящийся объект, иначе - исключение. Поскольку словари - изменяемый тип данных, используя эту нотацию, вы можете перезаписывать элементы словаря, добавлять новые, или удалять существующие с помощью встроенного оператора `del`:

In [24]:
my_dict = {'a': 1, 'b': 2}
print(my_dict['a'])

my_dict['a'] = [1, 2, 3]
print(my_dict)

my_dict['c'] = 1
print(my_dict)

del my_dict['b']
print(my_dict)


print(my_dict['b'])

1
{'a': [1, 2, 3], 'b': 2}
{'a': [1, 2, 3], 'b': 2, 'c': 1}
{'a': [1, 2, 3], 'c': 1}


KeyError: 'b'

**Итерируемость**:

Словари являются итерируемыми объектами. Итератор, созданный из словаря, на каждой итерации будет возвращать хранящийся в нем ключ:

In [25]:
my_dict = {'a': 1, 'b': 2}

for key in my_dict:
    print(key)

a
b


**Почему словари не последовательности**

Итак, почему же словари - не последовательности, ведь формально, словари реализуют интерфейс последовательностей: они итерируемы, они поддерживают вызов функции len(), а также доступ к элементами через квадратные скобочки? Словари не последовательности, поскольку словари - неупорядоченные колекции, что позволяет индексировать их элементы произвольными хэшируемыми объектами, в то время как последовательности являются упорядоченными коллекциями и индексируются исключительно целыми числами. 

### Методы словарей

Обсудим методы словарей и ситуации в которых они могут быть полезны.

**copy()**

Немодифицирующий метод словаря. Создает поверхностную копию словаря. Понятие поверхностной копии означает, что формально создается новый объект словаря, но при этом его содержимое - указатели на те же самые объекты, что и указатели в исходном словаре.

In [28]:
help(dict.copy)

Help on method_descriptor:

copy(...)
    D.copy() -> a shallow copy of D



In [26]:
my_dict = {'num': 1, 'list': [1, 2, 3]}
my_dict_copy = my_dict.copy()

print(
    f'original: {my_dict};',
    f'copy: {my_dict_copy};',
    id(my_dict) == id(my_dict_copy),
    sep='\n'
)

original: {'num': 1, 'list': [1, 2, 3]};
copy: {'num': 1, 'list': [1, 2, 3]};
False


In [27]:
my_dict_copy['list'].append(123)

print(
    f'original: {my_dict};',
    f'copy: {my_dict_copy};',
    sep='\n'
)

original: {'num': 1, 'list': [1, 2, 3, 123]};
copy: {'num': 1, 'list': [1, 2, 3, 123]};


**get()**

Метод get() является немодифицирующим методом.

In [29]:
help(dict.get)

Help on method_descriptor:

get(self, key, default=None, /)
    Return the value for key if key is in the dictionary, else default.



Метод get() может оказаться очень полезным для упрощения кода и избавления от ненужных ветвлений. Например, представим, что у нас есть некоторая функция, которая выполняет полезную работу, например итерационно оптимизирует некоторую функцию, т.е. старается за определенное число итераций найти минимум некоторой функции. Если нашей функции удается найти минимум за отведенное число итераций она возвращает нам статус и результат в виде словаря:

```Python
{
    "status": "success",
    "result": 42
}
```

Если же функции не удается найти минимум за отведенное число итераций, или в процессе выполнения происходит какая-либо ошибка, мы получаем ответ в формате:

```Python
{
    "status": "fail"
}
```

Так бы мог выглядит типичный код обработки ответа от нашей функции:

In [30]:
import random

from typing import Any


def simulate_optimization() -> dict[str, Any]:
    if random.randint(0, 1):
        return {
            "status": "success",
            "result": random.randint(-100, 100)
        }
    
    return {
        "status": "fail"
    }

In [34]:
result = simulate_optimization()

print(f'optimization status: {result["status"]}')

if "result" in result:
    print(f'optimization result: {result["result"]}')

else:
    print('optimization result: number of iterations exceeded')

optimization status: fail
optimization result: number of iterations exceeded


Но с посощью метода get() можно избавиться от ненужного ветвления:

In [36]:
result = simulate_optimization()

print(
    f'optimization status: {result["status"]}',
    f'optimization result: {result.get("result", "number of iterations exceeded")}',
    sep='\n'
)

optimization status: success
optimization result: -71


**items() / keys() / values()**

Метод items() является немодифицирующим методом.

In [37]:
help(dict.items)

Help on method_descriptor:

items(...)
    D.items() -> a set-like object providing a view on D's items



In [48]:
help(dict.keys)

Help on method_descriptor:

keys(...)
    D.keys() -> a set-like object providing a view on D's keys



In [49]:
help(dict.values)

Help on method_descriptor:

values(...)
    D.values() -> an object providing a view on D's values



In [50]:
my_dict = {'a': 'A', 'b': [1]}

In [51]:
for pair in my_dict.items():
    print(pair)

print('')

for key, value in my_dict.items():
    print(f'{key = }: {value = };')

('a', 'A')
('b', [1])

key = 'a': value = 'A';
key = 'b': value = [1];


In [53]:
for key in my_dict.keys():
    print(key)

print('')

for key in my_dict:
    print(key)

a
b

a
b


In [54]:
for val in my_dict.values():
    print(val)

A
[1]


Эти методы имеют много применений. Например, они могут использоваться для форматированного вывода содержимого словаря, как вы можете видеть это в примере сверху. Или для создания списка значений словаря.

**clear()**

Модифицирующий метод.

In [55]:
help(dict.clear)

Help on method_descriptor:

clear(...)
    D.clear() -> None.  Remove all items from D.



In [56]:
my_dict = {'a': 1, 'b': 2}
print(my_dict)

my_dict.clear()
print(my_dict)

{'a': 1, 'b': 2}
{}


**pop()**

Модифицирующий метод.

In [57]:
help(dict.pop)

Help on method_descriptor:

pop(...)
    D.pop(k[,d]) -> v, remove specified key and return the corresponding value.
    
    If the key is not found, return the default if given; otherwise,
    raise a KeyError.



In [58]:
my_dict = {'a': 1, 'b': 2}
print(my_dict)

popped_value = my_dict.pop('b')
print(f'poped value: {popped_value}; dict: {my_dict};')

popped_value = my_dict.pop('b', None)
print(f'poped value: {popped_value}; dict: {my_dict};')

my_dict.pop('b')

{'a': 1, 'b': 2}
poped value: 2; dict: {'a': 1};
poped value: None; dict: {'a': 1};


KeyError: 'b'

Данный метод также позволяет избежать использование ненужного ветвления.

Было:

In [61]:
my_dict ={'a': 1, 'b': 2, 'c': 3}

if 'c' in my_dict:
    value_c = my_dict['c']
    del my_dict['c']

else:
    value_c = None

print(f'dict["c"] value: {value_c}; dict: {my_dict}')

dict["c"] value: 3; dict: {'a': 1, 'b': 2}


**Стало:**

In [62]:
value_c = my_dict.pop('c', None)

print(f'dict["c"] value: {value_c}; dict: {my_dict}')

dict["c"] value: None; dict: {'a': 1, 'b': 2}


**popitem()**

Модифицирующий метод.

In [63]:
help(dict.popitem)

Help on method_descriptor:

popitem(self, /)
    Remove and return a (key, value) pair as a 2-tuple.
    
    Pairs are returned in LIFO (last-in, first-out) order.
    Raises KeyError if the dict is empty.



In [64]:
my_dict ={'a': 1, 'b': 2, 'c': 3}

while len(my_dict):
    key, value = my_dict.popitem()
    print(f'{key = }; {value = };')

key = 'c'; value = 3;
key = 'b'; value = 2;
key = 'a'; value = 1;


Данный метод может быть полезным, если вам необходимо перебрать все элементы словаря, хранение которого в памяти слишком затратны, произвести с ними некоторые действия, и удалить из словаря. 

**setdefault()**

Модифицирующий метод.

In [65]:
help(dict.setdefault)

Help on method_descriptor:

setdefault(self, key, default=None, /)
    Insert key with a value of default if key is not in the dictionary.
    
    Return the value for key if key is in the dictionary, else default.



In [68]:
my_dict = {'a': 1, 'b': 2}

value = my_dict.setdefault('a')
print(f'{value = }; dict: {my_dict};')

value = my_dict.setdefault('c')
print(f'{value = }; dict: {my_dict};')

value = my_dict.setdefault('d', 42)
print(f'{value = }; dict: {my_dict};')

value = 1; dict: {'a': 1, 'b': 2};
value = None; dict: {'a': 1, 'b': 2, 'c': None};
value = 42; dict: {'a': 1, 'b': 2, 'c': None, 'd': 42};


Этот метод также подойдет для оптимизации кода и отказа от использования ненужных ветвлений. Предположим, мы принимает на вход некоторую последовательность. Предположим, последовательность нуклиотидов. Наша задача определить позиции расположения каждого нуклеотида в этой последовательности.

**Неэффективный путь решения:**

In [75]:
def get_nucleatide_appearance(dna_sequence: str) -> dict[str, list[int]]:
    appearances = {}

    for i, nucleatide in enumerate(dna_sequence, start=1):
        nucleatide_appearances: list = appearances.get(nucleatide, [])
        nucleatide_appearances.append(i)
        appearances[nucleatide] = nucleatide_appearances

    return appearances

In [76]:
dna_sequence = 'agcccaagtat'

print(get_nucleatide_appearance(dna_sequence))

{'a': [1, 6, 7, 10], 'g': [2, 8], 'c': [3, 4, 5], 't': [9, 11]}


**Эффективный путь решения:**

In [77]:
def get_nucleatide_appearance(dna_sequence: str) -> dict[str, list[int]]:
    appearances = {}

    for i, nucleatide in enumerate(dna_sequence, start=1):
        appearances.setdefault(nucleatide, []).append(i)

    return appearances

In [78]:
dna_sequence = 'agcccaagtat'

print(get_nucleatide_appearance(dna_sequence))

{'a': [1, 6, 7, 10], 'g': [2, 8], 'c': [3, 4, 5], 't': [9, 11]}


**update()**

Модифицирующий метод.

In [79]:
help(dict.update)

Help on method_descriptor:

update(...)
    D.update([E, ]**F) -> None.  Update D from dict/iterable E and F.
    If E is present and has a .keys() method, then does:  for k in E: D[k] = E[k]
    If E is present and lacks a .keys() method, then does:  for k, v in E: D[k] = v
    In either case, this is followed by: for k in F:  D[k] = F[k]



In [84]:
my_dict = {'a': 1}

my_dict.update({'a': 42, 'b': 2})
my_dict.update([('c', 3), ('d', 4)])
my_dict.update(d=5)

print(my_dict)

{'a': 42, 'b': 2, 'c': 3, 'd': 5}


## Множества

Множества - непорядоченная коллекция уникальных элементов. Элементы множества обязательно должны быть хэшируемыми. В Python есть два типа множеств - `set` - изменяемое множество, - и `frozenset` - неизменяемое множество. Отсюда следует, что set не может являться элементом set'a, а frozenset - может. Все методы и операции frozenset - это немодифицирующие операции set'a, поэтому дальше мы будем обсуждать изменяемые множества, держа в голове, что их немодифицирующие методы применимы в отношении немодифицируемых множест. 

Основной сценарий использования множеств - определение уникальных элементов в итерируемом объекте. Также, за счет удобных операций над множествами, множества могут оказаться полезными в некоторых алгоритмах, использующих понятия непересекающихся множеств. Так, например, они могут быть полезными при реализации алгоритма Прима построения минимального остовного дерева.

Создание множеств:

In [85]:
sets = [
    {42, 3.14, 'hello'},
    {100},
    set(),
    frozenset([42, 3.14, 'hello']),
    frozenset([100]),
    frozenset()
]

for sets_item in sets:
    print(f'type: {type(sets_item).__name__}; value: {sets_item}')

type: set; value: {42, 3.14, 'hello'}
type: set; value: {100}
type: set; value: set()
type: frozenset; value: frozenset({42, 3.14, 'hello'})
type: frozenset; value: frozenset({100})
type: frozenset; value: frozenset()


### Методы множеств и операции над ними

Множества являются итерируемыми объектами. Они поддерживают вызов функции `len`, которая возвращает количество элементов множества, или, выражаясь математически, мощность множества. Также поддерживается вызов оператора `in`, который позволяет определить, принадлежит ли тот или иной объект данному множеству. 

**Немодифицирующие**

**copy()**

In [86]:
help(set.copy)

Help on method_descriptor:

copy(...)
    Return a shallow copy of a set.



In [89]:
my_set = {1, 'string'}
my_set_copy = my_set.copy()

print(
    f'original: {my_set};',
    f'copy: {my_set_copy};',
    id(my_set) == id(my_set_copy),
    sep='\n'
)

original: {1, 'string'};
copy: {1, 'string'};
False


**difference()**

In [92]:
help(set.difference)

Help on method_descriptor:

difference(...)
    Return the difference of two or more sets as a new set.
    
    (i.e. all elements that are in this set but not the others.)



In [100]:
set1 = set(range(10))
set2 = set(range(5))

print(set1.difference(set2))
print(set1 - set2)

print(set1.difference(set()))
print(set1.difference({11}))

{5, 6, 7, 8, 9}
{5, 6, 7, 8, 9}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


**intersection()**

In [95]:
help(set.intersection)

Help on method_descriptor:

intersection(...)
    Return the intersection of two sets as a new set.
    
    (i.e. all elements that are in both sets.)



In [98]:
set1 = set(range(10))
set2 = set(range(5))

print(set1.intersection(set2))
print(set1 & set2)

print(set1.intersection(set()))
print(set1.intersection({11}))

{0, 1, 2, 3, 4}
{0, 1, 2, 3, 4}
set()
set()


**issubset**

In [101]:
help(set.issubset)

Help on method_descriptor:

issubset(...)
    Report whether another set contains this set.



In [108]:
set1 = set(range(10))
set2 = set(range(5))

print(set1.issubset(set2))
print(set1 <= set2)

print(set1.issubset(set()))

False
False
False


**issuperset**

In [105]:
help(set.issuperset)

Help on method_descriptor:

issuperset(...)
    Report whether this set contains another set.



In [107]:
set1 = set(range(10))
set2 = set(range(5))

print(set1.issuperset(set2))
print(set1 >= set2)

print(set1.issuperset(set()))

True
True
True


**union**

In [113]:
help(set.union)

Help on method_descriptor:

union(...)
    Return the union of sets as a new set.
    
    (i.e. all elements that are in either set.)



In [114]:
set1 = set(range(10))
set2 = set(range(5))

print(set1.union(set2))
print(set1 | set2)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


**symmetric_difference()**

In [109]:
help(set.symmetric_difference)

Help on method_descriptor:

symmetric_difference(...)
    Return the symmetric difference of two sets as a new set.
    
    (i.e. all elements that are in exactly one of the sets.)



In [112]:
set1 = set(range(10)) | set(range(13, 16))
set2 = set(range(5)) | set(range(10, 15))

print(set1.symmetric_difference(set2))
print(set1 ^ set2)

{5, 6, 7, 8, 9, 10, 11, 12, 15}
{5, 6, 7, 8, 9, 10, 11, 12, 15}


**Модифицирующие**

Все операторы, результатами операций с которыми являются новые множества, могут образовывать операторы составного присваивания: `|=`, `-=`, `|=`, `&=`. Несложно догадаться, что именно делают эти операторы. 

**add()**

In [116]:
help(set.add)

Help on method_descriptor:

add(...)
    Add an element to a set.
    
    This has no effect if the element is already present.



In [117]:
my_set = {1, 2, 3}

my_set.add(3)
my_set.add('a')

print(my_set)

my_set.add([1, 2, 3])

{1, 2, 3, 'a'}


TypeError: unhashable type: 'list'

**clear()**

In [118]:
my_set = {1, 2, 3}
my_set.clear()

print(my_set)

set()


**discard()**

In [119]:
help(set.discard)

Help on method_descriptor:

discard(...)
    Remove an element from a set if it is a member.
    
    Unlike set.remove(), the discard() method does not raise
    an exception when an element is missing from the set.



In [120]:
my_set = {1, 2, 3}
my_set.discard(1)
my_set.discard(42)

print(my_set)

{2, 3}


**pop()**

In [121]:
help(set.pop)

Help on method_descriptor:

pop(...)
    Remove and return an arbitrary set element.
    Raises KeyError if the set is empty.



In [122]:
my_set = {1, 2, 3}

while len(my_set):
    print(f'popped value: {my_set.pop()}')

popped value: 1
popped value: 2
popped value: 3


In [123]:
my_set.pop()

KeyError: 'pop from an empty set'

**remove()**

In [124]:
my_set = {1, 2, 3}
my_set.remove(1)
print(my_set)

my_set.remove(42)

{2, 3}


KeyError: 42